# Download de Internações por Doença Respiratória (CID-10 J) - Rio de Janeiro
Este notebook realiza o download dos dados de internação hospitalar por doenças respiratórias (CID-10 iniciando com a letra **J**) na cidade do Rio de Janeiro, utilizando a biblioteca **PySUS**.

**Observações importantes:**
- O download utilizando a biblioteca **PySUS** só é possível em **Linux** ou no **Google Colab**.


## Instalação de dependências
Instala a biblioteca **PySUS** para acessar os dados do DataSUS.

In [1]:
!pip install pysus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

## Importação de bibliotecas

Carrega as bibliotecas necessárias para manipulação e download dos dados.

In [2]:
from pysus import SIH
import pandas as pd
import os
from zipfile import ZipFile

## Download e filtro dos dados
Esta função automatiza o processo de download, filtragem e organização dos dados de internações hospitalares (SIH) para o Rio de Janeiro.

**Funcionalidades:**

*   **Seleção de Período:** Permite especificar anos e meses para download.
*   **Download de Dados:** Baixa os arquivos de dados do SIH para o período e UF (RJ) especificados.
*   **Filtragem por CID-10:** Filtra os registros, mantendo apenas as internações cujo diagnóstico principal (DIAG_PRINC) inicia com 'J' (doenças respiratórias) e identificador '1'.
*   **Seleção de colunas:** Seleciona as colunas mais relevantes para análise.
*   **Formatação de data:** Converte a data de internação para o formato AAAA-MM-DD e filtra os dados entre 2012 e 2024.
*   **Salvamento Anual:** Salva os dados filtrados de cada ano em um arquivo CSV separado.

In [ ]:
def download_and_filter_data(anos, meses=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]):

    if isinstance(anos, int):
        anos = [anos]

    df_concatenado = pd.DataFrame()

    for ano in anos:
      for mes in meses:
        # Baixar os arquivos para o ano e mês específico
        sih = SIH().load()
        files = sih.get_files("RD", uf="RJ", year=ano, month=[mes])

        # Baixar os dados dos arquivos
        parquet = sih.download(files)

        # Converter os dados para um DataFrame
        df = parquet.to_dataframe()

        # Filtrar a coluna 'DIAG_PRINC' conforme os valores especificados
        df_filtrado = df[(df['DIAG_PRINC'].str.startswith('J')) & (df['IDENT'] == '1')]

        # Selecionando as colunas mais relevantes
        interest_columns = ['UF_ZI', 'ANO_CMPT', 'MES_CMPT', 'MUNIC_RES', 'MUNIC_MOV', 'CEP', 'NASC', 'SEXO', 'DT_INTER', 'DT_SAIDA', 'DIAG_PRINC', 'DIAG_SECUN', 'IDADE', 'COD_IDADE', 'RACA_COR', 'ETNIA', 'DIAS_PERM', 'MORTE', 'CID_MORTE']

        df_filtrado = df_filtrado[interest_columns]

        # Concatenar os dados do mês com os dados já baixados
        df_concatenado = pd.concat([df_concatenado, df_filtrado], ignore_index=True)

        print(f'Dados do ano {ano} - mês {mes} baixados e filtrados.')

      df_concatenado['data_formatada'] = pd.to_datetime(df_concatenado['DT_INTER'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

      df_concatenado.sort_values(by='data_formatada', inplace=True)

      df_concatenado = df_concatenado[(df_concatenado['data_formatada'] >= '2012-01-01') & (df_concatenado['data_formatada'] <= '2024-12-31')]

      df_concatenado.to_csv(f'dados_filtrados_{ano}.csv', index=False)

    print(f'Dados do(s) ano(s) {anos} salvos com sucesso em "dados_filtrados_{anos[0]}_a_{anos[-1]}.csv".')

## Execução do download

Baixa os dados de 2012 até 2024 e aplica o filtro.

In [ ]:
download_and_filter_data(anos=list(range(2012, 2025)))

RDRJ1201.parquet: 100%|██████████| 172k/172k [00:09<00:00, 18.4kB/s]


Dados do ano 2012 - mês 1 baixados e filtrados.


RDRJ1202.parquet: 100%|██████████| 168k/168k [00:09<00:00, 18.0kB/s]


Dados do ano 2012 - mês 2 baixados e filtrados.


RDRJ1203.parquet: 100%|██████████| 199k/199k [00:11<00:00, 17.5kB/s]


...

RDRJ1509.parquet: 100%|██████████| 210k/210k [00:13<00:00, 15.3kB/s]


Dados do ano 2015 - mês 9 baixados e filtrados.


RDRJ1510.parquet: 100%|██████████| 212k/212k [00:15<00:00, 14.0kB/s]


Dados do ano 2015 - mês 10 baixados e filtrados.


RDRJ1511.parquet:   0%|          | 0.00/203k [00:00<?, ?B/s]